In [0]:
#LOAD DATASET 
customers = spark.table("default.customer_transactions")
customers.show(5)


+-----------+---+------+---------+-------------+--------------+-------------+------------------+----------------+------------------------+-----------------+-------+--------------------+------------------+
|customer_id|age|gender|  country|annual_income|spending_score|num_purchases|avg_purchase_value|membership_years|website_visits_per_month|cart_abandon_rate|churned|       feedback_text|last_purchase_date|
+-----------+---+------+---------+-------------+--------------+-------------+------------------+----------------+------------------------+-----------------+-------+--------------------+------------------+
|          1| 37|  Male|  Germany|        85886|            14|           18|              41.2|               6|                      20|             0.95|      0|Very satisfied wi...|        2025-06-22|
|          2| 40|  Male|    India|        41041|             4|           10|             31.73|               4|                      29|             0.21|      0|Good quality and

In [0]:
#PySpark vs Pandas (small demo)
customers.limit(5).toPandas()


,customer_id,age,gender,country,annual_income,spending_score,num_purchases,avg_purchase_value,membership_years,website_visits_per_month,cart_abandon_rate,churned,feedback_text,last_purchase_date
0,1,37,Male,Germany,85886,14,18,41.20,6,20,0.95,0,Very satisfied with my purchase.,2025-06-22
1,2,40,Male,India,41041,4,10,31.73,4,29,0.21,0,Good quality and value for money.,2025-10-17
2,3,69,Female,Australia,143869,59,39,65.96,12,26,0.08,0,Excellent customer service.,2025-07-01
3,4,30,Male,UK,87261,45,34,51.87,12,7,0.61,0,Good quality and value for money.,2025-08-17
4,5,69,Female,UK,110678,40,38,59.64,13,16,0.49,0,Excellent customer service.,2025-06-21


In [0]:
#Create a small lookup table:
country_region = spark.createDataFrame([
    ("India", "Asia"),
    ("Germany", "Europe"),
    ("USA", "North America"),
    ("UK", "Europe"),
    ("Australia", "Oceania")
], ["country", "region"])


In [0]:
country_region.show(5)

+---------+-------------+
|  country|       region|
+---------+-------------+
|    India|         Asia|
|  Germany|       Europe|
|      USA|North America|
|       UK|       Europe|
|Australia|      Oceania|
+---------+-------------+



In [0]:
#JOIN IT 
joined_data = customers.join(
    country_region,
    on="country",
    how="inner"
)

joined_data.show(5)

+---------+-----------+---+------+-------------+--------------+-------------+------------------+----------------+------------------------+-----------------+-------+--------------------+------------------+-------+
|  country|customer_id|age|gender|annual_income|spending_score|num_purchases|avg_purchase_value|membership_years|website_visits_per_month|cart_abandon_rate|churned|       feedback_text|last_purchase_date| region|
+---------+-----------+---+------+-------------+--------------+-------------+------------------+----------------+------------------------+-----------------+-------+--------------------+------------------+-------+
|  Germany|          1| 37|  Male|        85886|            14|           18|              41.2|               6|                      20|             0.95|      0|Very satisfied wi...|        2025-06-22| Europe|
|    India|          2| 40|  Male|        41041|             4|           10|             31.73|               4|                      29|          

In [0]:
#WINDOW FUNCTION
from pyspark.sql.window import Window
from pyspark.sql import functions as F

window_spec = Window.partitionBy("customer_id") \
                    .orderBy("last_purchase_date")

customers = customers.withColumn(
    "running_purchases",
    F.sum("num_purchases").over(window_spec)
)

customers.select(
    "customer_id", "num_purchases", "running_purchases"
).show(5)

+-----------+-------------+-----------------+
|customer_id|num_purchases|running_purchases|
+-----------+-------------+-----------------+
|          1|           18|               18|
|          2|           10|               10|
|          3|           39|               39|
|          4|           34|               34|
|          5|           38|               38|
+-----------+-------------+-----------------+
only showing top 5 rows


In [0]:
#RANK CUSTOMERS BY INCOME
rank_window = Window.partitionBy("country") \
                    .orderBy(F.desc("annual_income"))

customers = customers.withColumn(
    "income_rank",
    F.rank().over(rank_window)
)

customers.select(
    "customer_id", "country", "annual_income", "income_rank"
).show(5)


+-----------+---------+-------------+-----------+
|customer_id|  country|annual_income|income_rank|
+-----------+---------+-------------+-----------+
|       3853|Australia|       179713|          1|
|       8192|Australia|       179334|          2|
|       8631|Australia|       179252|          3|
|       1866|Australia|       178988|          4|
|       4657|Australia|       178878|          5|
+-----------+---------+-------------+-----------+
only showing top 5 rows


In [0]:
customers = customers.withColumn(
    "customer_value",
    F.col("avg_purchase_value") * F.col("num_purchases")
)

customers = customers.withColumn(
    "engagement_score",
    F.col("website_visits_per_month") + F.col("num_purchases")
)

customers.show(5)

+-----------+---+------+---------+-------------+--------------+-------------+------------------+----------------+------------------------+-----------------+-------+--------------------+------------------+-----------------+-----------+--------------+----------------+
|customer_id|age|gender|  country|annual_income|spending_score|num_purchases|avg_purchase_value|membership_years|website_visits_per_month|cart_abandon_rate|churned|       feedback_text|last_purchase_date|running_purchases|income_rank|customer_value|engagement_score|
+-----------+---+------+---------+-------------+--------------+-------------+------------------+----------------+------------------------+-----------------+-------+--------------------+------------------+-----------------+-----------+--------------+----------------+
|       3853| 46|Female|Australia|       179713|            30|           33|             69.92|              11|                      22|              0.7|      0|The website is ea...|        2025-0

In [0]:
customers.groupBy("country") \
    .agg(
        F.avg("customer_value").alias("avg_customer_value"),
        F.avg("engagement_score").alias("avg_engagement")
    ) \
    .orderBy(F.desc("avg_customer_value")) \
    .show()


+------------+------------------+------------------+
|     country|avg_customer_value|    avg_engagement|
+------------+------------------+------------------+
|   Australia| 1255.691652046784|39.198830409356724|
|       Japan|1234.3136565656566| 38.99393939393939|
|         USA| 1233.473624947677| 39.02553369610716|
|      Canada|1227.1785990338158|38.666666666666664|
|     Germany| 1222.493869047619| 39.20039682539682|
|          UK| 1215.137581441263| 38.72359328726555|
|South Africa| 893.0692385786808|36.203045685279186|
|       India| 889.0494750656184|36.808398950131235|
|      Brazil| 885.1380780487797| 36.49170731707317|
|      France| 868.3905781249998|         36.990625|
+------------+------------------+------------------+

